In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from paths import paths

Decision Logic:

    if unilateral: L
    elif bilateral:
        if not synchronous: L
        elif not symmetric: L
        else: G

In [26]:
df = pd.read_csv(paths.TRAIN_CSV)
label_cols = df.columns[-6:]
number_experts = df[label_cols].sum(axis = 1)
for label in label_cols:
    df[label] = df[label] / number_experts
display(df)

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0


In [27]:
df['G'] = df['gpd_vote'] + df['grda_vote']
df['L'] = df['lpd_vote'] + df['lrda_vote']

useful_cols = ['eeg_id', 'eeg_sub_id', 'eeg_label_offset_seconds', 'G', 'L']
dataframe = df[useful_cols]
display(dataframe)

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,G,L
0,1628180742,0,0.0,0.0,0.0
1,1628180742,1,6.0,0.0,0.0
2,1628180742,2,8.0,0.0,0.0
3,1628180742,3,18.0,0.0,0.0
4,1628180742,4,24.0,0.0,0.0
...,...,...,...,...,...
106795,351917269,6,12.0,0.0,1.0
106796,351917269,7,14.0,0.0,1.0
106797,351917269,8,16.0,0.0,1.0
106798,351917269,9,18.0,0.0,1.0


In [28]:
gl_df = dataframe.loc[dataframe['G'] + dataframe['L'] == 1]
display(gl_df)

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,G,L
22,387987538,0,0.0,0.0,1.0
23,387987538,1,2.0,0.0,1.0
24,387987538,2,6.0,0.0,1.0
25,387987538,3,8.0,0.0,1.0
26,387987538,4,10.0,0.0,1.0
...,...,...,...,...,...
106795,351917269,6,12.0,0.0,1.0
106796,351917269,7,14.0,0.0,1.0
106797,351917269,8,16.0,0.0,1.0
106798,351917269,9,18.0,0.0,1.0


In [29]:
gl_df.index[0]

22

In [71]:
from sklearn.linear_model import LinearRegression

def normalize(array):
    centered = np.array(array) - np.mean(array)
    std = np.std(centered)
    if std == 0:
        return centered
    return pd.Series(centered / np.std(centered))

def four_way(example):
    ret = pd.DataFrame()
    for region in paths.BRAIN_REGIONS.keys():
        for col in paths.BRAIN_REGIONS[region]:
            if region not in ret.columns:
                ret[region] = example[col]
            else: ret[region] = ret[region] + example[col]
    return ret

def get_predictiveness(example):
    fourway = four_way(example)
    normalized = fourway.apply(normalize, axis=1)
    linear_shift = LinearRegression().fit(normalized[:-2], normalized.shift(periods=-2)[:-2])
    predictive_power = np.mean(linear_shift.coef_, axis=1)
    #predictiveness = normalize(predictive_power)
    predictiveness = pd.Series(predictive_power)
    predictiveness.index = paths.BRAIN_REGIONS.keys()
    return predictiveness

def predictiveness(id, offset):
    example = pd.DataFrame(np.load(paths.TRAIN_CLEAN_10 + id
                                   + '.npy'))[20*offset:20*(offset+50)]
    example.columns = paths.EEG_LABELS
    return get_predictiveness(example)

In [72]:
example = pd.DataFrame(np.load(paths.TRAIN_CLEAN_10 + '387987538.npy'))[:20*50]
example.columns = paths.EEG_LABELS
display(get_predictiveness(example))

LL    17644.837891
LP    31362.763672
RP    15461.128906
RR   -64468.937500
dtype: float32

In [73]:
predictivities = {}
for i, idx in enumerate(tqdm(gl_df.index)):
    if i > 199: break
    id = str(gl_df.loc[idx,'eeg_id'])
    offset = int(gl_df.loc[idx,'eeg_label_offset_seconds'])
    predictivities[idx] = predictiveness(id, offset)

  0%|          | 0/33160 [00:00<?, ?it/s]

In [57]:
predictivity_df = pd.DataFrame(predictivities).transpose()
display(predictivity_df)
total_df = predictivity_df.join(gl_df[['G','L']], how='left')
display(total_df)

,LL,LP,RP,RR
22,1.186082,-0.184088,0.513233,-1.515227
23,0.924654,0.691589,0.016681,-1.632924
24,0.896292,-1.162338,1.084401,-0.818355
25,1.445621,-0.315236,0.200289,-1.330673
26,1.219787,0.266108,0.074782,-1.560678
...,...,...,...,...
1218,1.218897,0.625190,-1.381988,-0.462100
1219,1.584676,-0.198711,-1.187385,-0.198580
1220,-0.492080,0.830736,-1.396192,1.057537
1221,-1.086408,1.199266,-0.885618,0.772761


,LL,LP,RP,RR,G,L
22,1.186082,-0.184088,0.513233,-1.515227,0.0,1.0
23,0.924654,0.691589,0.016681,-1.632924,0.0,1.0
24,0.896292,-1.162338,1.084401,-0.818355,0.0,1.0
25,1.445621,-0.315236,0.200289,-1.330673,0.0,1.0
26,1.219787,0.266108,0.074782,-1.560678,0.0,1.0
...,...,...,...,...,...,...
1218,1.218897,0.625190,-1.381988,-0.462100,0.0,1.0
1219,1.584676,-0.198711,-1.187385,-0.198580,0.0,1.0
1220,-0.492080,0.830736,-1.396192,1.057537,0.0,1.0
1221,-1.086408,1.199266,-0.885618,0.772761,0.0,1.0


In [62]:
total_G = total_df.loc[total_df['G']==1]
total_L = total_df.loc[total_df['L']==1]
display(total_G.describe())
total_L.describe()

,LL,LP,RP,RR,G,L
count,163.000000,163.000000,163.000000,163.000000,163.0,163.0
mean,0.408799,-0.066929,0.014766,-0.356635,1.0,0.0
std,1.063420,0.899939,0.923452,0.964535,0.0,0.0
min,-1.652152,-1.667604,-1.662167,-1.721462,1.0,0.0
25%,-0.572989,-0.814133,-0.666725,-1.172294,1.0,0.0
50%,0.715223,-0.081524,-0.224576,-0.622949,1.0,0.0
75%,1.437031,0.648537,0.937210,0.472348,1.0,0.0
max,1.723507,1.715271,1.653781,1.692456,1.0,0.0


,LL,LP,RP,RR,G,L
count,121.000000,121.000000,121.000000,121.000000,121.0,121.0
mean,-0.134421,-0.116437,0.267475,-0.016616,0.0,1.0
std,0.936700,0.994639,0.941095,1.084736,0.0,0.0
min,-1.543768,-1.731383,-1.428784,-1.696103,0.0,1.0
25%,-0.951892,-0.925955,-0.636075,-0.979493,0.0,1.0
50%,-0.302212,-0.375050,0.463798,-0.089929,0.0,1.0
75%,0.733265,0.830736,1.090844,0.959361,0.0,1.0
max,1.584676,1.690978,1.713886,1.671605,0.0,1.0
